In [1]:
import numpy as np
import pandas as pd

In [2]:
train_df = pd.read_csv("train.csv")

train_df.head()

,id,url,title,target
0,0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц...",False
1,1,www.kp.by,Эта песня стала известна многим телезрителям б...,False
2,2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...,False
3,3,colorbox.spb.ru,Не Беси Меня Картинки,False
4,4,tula-sport.ru,В Новомосковске сыграют следж-хоккеисты алекси...,False


In [3]:
test_df = pd.read_csv("test.csv")

test_df.head()

,id,url,title
0,135309,www.kommersant.ru,Шестой кассационный суд в Самаре начнет работу...
1,135310,urexpert.online,"Что такое индексация алиментов, кем и в каких ..."
2,135311,imperimeha.ru,Женщинам | Империя Меха - Part 12
3,135312,national-porn.com,"Небритые, волосатые киски: Порно всех стран и ..."
4,135313,2gis.ru,67


In [4]:
len(test_df)

165378

In [5]:
train_df["target"].value_counts()

False    118594
True      16715
Name: target, dtype: int64

In [6]:
from sklearn.metrics import f1_score

In [7]:
X_train = train_df["title"].values
X_test = test_df["title"].values
y_train = train_df["target"].astype(int).values

# Simple baseline

In [8]:
y_pred = [int("порно" in text) for text in X_train]

In [9]:
f1_score(y_train, y_pred)

0.31242758551206484

### Submit

In [10]:
test_df["target"] = [("порно" in text) for text in X_test]

test_df[["id", "target"]].to_csv("simple_baseline.csv", index=False)

!cat simple_baseline.csv | head

"cat" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


### Не все так однозначно

**не порно**:
- Болезни опорно-двигательной системы и импотенция: взаимосвязь
- Транссексуальные рыбы - National Geographic Россия: красота мира в каждом кадре
- Групповая обзорная экскурсия по Афинам - цена €50
- Больного раком Задорнова затравили в соцсетях.
- Гомосексуалисты на «Первом канале»? Эрнст и Галкин – скрытая гей-пара российского шоу-бизнеса | Заметки о стиле, моде и жизни

**порно**:
- Отборная домашка
- Сюзанна - карьера горничной / Susanna cameriera perversa (с русским переводом) 1995 г., DVDRip

# ML baseline

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [12]:
train_df.head(40)

,id,url,title,target
0,0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц...",False
1,1,www.kp.by,Эта песня стала известна многим телезрителям б...,False
2,2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...,False
3,3,colorbox.spb.ru,Не Беси Меня Картинки,False
4,4,tula-sport.ru,В Новомосковске сыграют следж-хоккеисты алекси...,False
5,5,beregifiguru.ru,Салат: корейская морковь + копченая курица + к...,False
6,6,ekb.vseinstrumenti.ru,"Угловой пневмогайковерт FUBAG RWC 105 1/2"" 100...",False
7,7,mirtabaka.org,Табачный магазин `Мир Табака` | КУРИТЕЛЬНЫЕ ТР...,False
8,8,xlecx.com,league of legends » Page 5 » Porn comics free ...,True
9,9,bus.biletyplus.ua,"08:00 (4766р), Одесса - Ростов-на-Дону 20 сент...",False


In [13]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [15]:
train_df.title[2]

'Банши 4 сезон 2 серия Бремя красоты смотреть онлайн!'

In [16]:
y_train = train_df["target"].astype(int).values

In [17]:
import re

regex = re.compile(u"[А-Яа-яA-Za-z]+")

def words_only(text, regex=regex):
    return " ".join(regex.findall(text))


train_df.title = train_df.title.str.lower()
train_df.loc[:, 'title'] = train_df.title.apply(words_only)
test_df.title = test_df.title.str.lower()
test_df.loc[:, 'title'] = test_df.title.apply(words_only)

In [18]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mdime\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
# Удаляем стоп-слова
mystopwords = stopwords.words('russian') + stopwords.words('english') + ['это', 'наш' , 'млн', 'млрд', u'также',  'т', 'д', '-', 'net', 'ru']

def  remove_stopwords(text, mystopwords = mystopwords):
    try:
        return u" ".join([token for token in text.split() if not token in mystopwords])
    except:
        return u""

test_df.title = test_df.title.apply(remove_stopwords)
train_df.title = train_df.title.apply(remove_stopwords)

In [19]:
mystoplemmas = [u'который', u'прошлый', u'сей', u'свой', u'наш', u'мочь']

# Еще кое-что удаляем
def  remove_stoplemmas(text, mystoplemmas = mystoplemmas):
    try:
        return " ".join([token for token in text.split() if not token in mystoplemmas])
    except:
        return ""

test_df.title = test_df.title.apply(remove_stoplemmas)
train_df.title = train_df.title.apply(remove_stoplemmas) 

In [20]:
train_df.title[2]

'банши сезон серия бремя красоты смотреть онлайн'

In [21]:
X_train = train_df["title"].values
X_test = test_df["title"].values

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB

In [31]:
vectorizer2 = TfidfVectorizer()
X_train_vectorized2 = vectorizer2.fit_transform(X_train)
model2 = MultinomialNB(alpha = 1.5)

model2.fit(X_train_vectorized2, y_train)
y_pred = model2.predict(X_train_vectorized2)

print(f1_score(y_train, y_pred))

0.9320880164635111


In [106]:
X_test_vectorized = vectorizer2.transform(X_test)

test1_df["target"] = model2.predict(X_test_vectorized).astype(bool)

test1_df[["id", "target"]].to_csv("prediction2.csv", index=False)

!cat prediction2.csv | head

"cat" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.
